In [1]:
# Load metrics
#from sklearn.metrics import roc_auc_score

# Description: Load tsv file

# Import libraries
import pandas as pd
import random


# Load tsv file
News_vali = pd.read_csv('MINDsmall_dev/news.tsv', sep='\t', header=None)
News_vali.columns = ['news_id', 'category', 'subcategory', 'title', 'abstract', 'url', 'title_entities', 'abstract_entities']

User_vali = pd.read_csv('MINDsmall_dev/behaviors.tsv', sep='\t', header=None)
User_vali.columns = ['impression_id', 'user_id', 'time', 'history', 'impressions']



In [2]:
# Description: Load tsv file

# Import libraries
import pandas as pd
import random

# Load tsv file
News = pd.read_csv('MINDsmall_train/news.tsv', sep='\t', header=None)
News.columns = ['news_id', 'category', 'subcategory', 'title', 'abstract', 'url', 'title_entities', 'abstract_entities']
News_vali = pd.read_csv('MINDsmall_dev/news.tsv', sep='\t', header=None)
News_vali.columns = ['news_id', 'category', 'subcategory', 'title', 'abstract', 'url', 'title_entities', 'abstract_entities']

News_con = pd.concat([News, News_vali], ignore_index=True)


UserData = pd.read_csv('MINDsmall_train/behaviors.tsv', sep='\t', header=None)
UserData.columns = ['impression_id', 'user_id', 'time', 'history', 'impressions']

UserData = UserData.dropna()

topic_size = News['category'].nunique()
subtopic_size = News['subcategory'].nunique()

print(f"Data contains {topic_size} topics and {subtopic_size} subtopics")

Data contains 17 topics and 264 subtopics


In [3]:
# Define Vocabulary for users and topics
from torchtext import vocab
from torchtext.data.utils import get_tokenizer
import torch as th
from LSTUR import GloVe

tokenizer = get_tokenizer('basic_english')

User_vocab = vocab.build_vocab_from_iterator([[id] for id in UserData['user_id']], specials=['<unk>'])
User_vocab.set_default_index(User_vocab['<unk>'])
News_vocab = vocab.build_vocab_from_iterator([[id] for id in  News_con['news_id']], specials=['<unk>'])
News_vocab.set_default_index(News_vocab['<unk>'])
Category_vocab = vocab.build_vocab_from_iterator([[Category] for Category in News['category']], specials=['<unk>'])
Category_vocab.set_default_index(Category_vocab['<unk>'])
Subcategory_vocab = vocab.build_vocab_from_iterator([[Category] for Category in News['subcategory']], specials=['<unk>'])
Subcategory_vocab.set_default_index(Subcategory_vocab['<unk>'])

/Users/magnusharder/Documents/UNI-DTU/6. Semester/Bachelor Projekt/News-Recommendations/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Define Vocabulary for title and abstract
max_title_length = max([len(tokenizer(title)) for title in News['title']])
max_history_length = max([len(history.split(" ")) for history in UserData['history']])
max_history_length = 50 # Overwrite

impressions_length = max([len(impressions.split(" ")) for impressions in User_vali['impressions']])
#max_impressions_length = 5 # Overwrite

# Define Datapoint to tensor
def Datapoint_to_Encodings(User):

    History = News_vocab.lookup_indices(User.history.split(" "))
    User_en = User_vocab.__getitem__(User.user_id)
    Impressions = User.impressions.split(" ")
    Impressions,Clicked = map(list, zip(*[Impression.split("-") for Impression in Impressions]))
    
    Positive, Negative = [],[]
    for idx, click in enumerate(Clicked):
        if click == "1":
            Positive.append(Impressions[idx])
        else:
            Negative.append(Impressions[idx])

    Impressions = [Positive[0]]

   

    if len(Negative) > 3:
        for _ in random.sample(Negative,4):
            Impressions.append(_)
    else:
        for _ in range(4):
            Impressions.append(random.choice(Negative))

    Clicked = [1,0,0,0,0]

    # Shuffle
    shuffled_index = [0,1,2,3,4]
    random.shuffle(shuffled_index)


    Impressions = [Impressions[i] for i in shuffled_index]
    Clicked = [Clicked[i] for i in shuffled_index]


    # Convert to tensor
    Impressions = News_vocab.lookup_indices(Impressions)
    History, User_en, Impressions, Clicked = map(th.tensor, [History, User_en, Impressions, Clicked])

    return History, User_en, Impressions, Clicked

# Define Datapoint to tensor
def Datapoint_to_Encodings_vali(User):

    History = News_vocab.lookup_indices(User.history.split(" "))
    User_en = User_vocab.__getitem__(User.user_id)
    Impressions = User.impressions.split(" ")
    Impressions,Clicked = map(list, zip(*[Impression.split("-") for Impression in Impressions]))
    

    # Convert to tensor
    Impressions = News_vocab.lookup_indices(Impressions)
    Clicked = [int(click) for click in Clicked]

    History, User_en, Impressions, Clicked = map(th.tensor, [History, User_en, Impressions, Clicked])

    return History, User_en, Impressions, Clicked

# Pack Title
def pack_Title(title,max_length):

    src_len, _ = title.size()

    title_reformated = th.zeros(max_length,300)

    title_reformated[:src_len,:] = title

    return title_reformated, src_len


# Get Numeric Artikles representation
def get_Article_Encodings(Artikle):


    title = GloVe.get_vecs_by_tokens(tokenizer(Artikle['title']))
    
    #Abstract = [tokenizer(abstract) for abstract in Artikle['abstract']]
    Category = Category_vocab.__getitem__(Artikle['category'])
    Subcategory = Subcategory_vocab.__getitem__(Artikle['subcategory'])

    title, title_len = pack_Title(title,max_title_length)

    Category, Subcategory, title_len = map(th.tensor, [Category, Subcategory, title_len])

    

    return Category, Subcategory, title, title_len

# Store all News in Dictionary for faster access
News_tensors = {}

for i in range(len(News_con)):
    News_tensors[News_vocab.__getitem__(News_con['news_id'][i])] = get_Article_Encodings(News_con.loc[i])

# Get Numeric User representation
def Datapoint_to_tensor(User,train=True):

    if train:
        History, User_en, Impressions, Clicked = Datapoint_to_Encodings(User)
        max_impressions_length = 5
        if random.random() < 0.4:
            User_en = 0 # Mask user 
    else:
        max_impressions_length = impressions_length
        History, User_en, Impressions, Clicked = Datapoint_to_Encodings_vali(User)



    History_tensor = th.zeros(max_history_length,max_title_length,300)
    Category = th.zeros(max_history_length)
    Subcategory = th.zeros(max_history_length)
    history_len = min(len(History),max_history_length)

    for idx,article in enumerate(History[-history_len:]):
        Category[idx], Subcategory[idx], History_tensor[idx], _ = News_tensors[article.item()]

    Impressions_tensor = th.zeros(max_impressions_length,max_title_length,300)
    Category_Impressions = th.zeros(max_impressions_length)
    Subcategory_Impressions = th.zeros(max_impressions_length)
    Impressions_len = len(Impressions)

    history_len, Impressions_len = map(th.tensor, [history_len, Impressions_len])


    for idx,article in enumerate(Impressions):
        Category_Impressions[idx], Subcategory_Impressions[idx], Impressions_tensor[idx], _ = News_tensors[article.item()]
    
    Clicked = Clicked.argmax()

    return User_en, Category, Subcategory, History_tensor, history_len, Category_Impressions, Subcategory_Impressions, Impressions_tensor, Impressions_len, Clicked


# Def load batch
def load_batch(User, batch_size, device='cpu',train=True):
    
        #User = User.sample(frac=1).reset_index(drop=True)
    
        for i in range(0, len(User), batch_size):
    
            User_batch = User[i:i+batch_size]
    
            User_en = []
            Category = []
            Subcategory = []
            History_tensor = []
            history_len = []
            Category_Impressions = []
            Subcategory_Impressions = []
            Impressions_tensor = []
            Impressions_len = []
            Clicked = []
    
            for i in range(len(User_batch)):
                User_en_, Category_, Subcategory_, History_tensor_, history_len_, Category_Impressions_, Subcategory_Impressions_, Impressions_tensor_, Impressions_len_, Clicked_ = Datapoint_to_tensor(User_batch.iloc[i],train=train)
                User_en.append(User_en_)
                Category.append(Category_)
                Subcategory.append(Subcategory_)
                History_tensor.append(History_tensor_)
                history_len.append(history_len_)
                Category_Impressions.append(Category_Impressions_)
                Subcategory_Impressions.append(Subcategory_Impressions_)
                Impressions_tensor.append(Impressions_tensor_)
                Impressions_len.append(Impressions_len_)
                Clicked.append(Clicked_)
    
            User_en, Category, Subcategory, History_tensor, history_len, Category_Impressions, Subcategory_Impressions, Impressions_tensor, Impressions_len, Clicked = map(th.stack, [User_en, Category, Subcategory, History_tensor, history_len, Category_Impressions, Subcategory_Impressions, Impressions_tensor, Impressions_len, Clicked])
            User_en, Category, Subcategory, history_len, Category_Impressions, Subcategory_Impressions, Impressions_len, Clicked = map(lambda x: x.long(), [User_en, Category, Subcategory, history_len, Category_Impressions, Subcategory_Impressions, Impressions_len, Clicked])
            yield User_en.to(device), Category.to(device), Subcategory.to(device), History_tensor.to(device), history_len.to(device), Category_Impressions.to(device), Subcategory_Impressions.to(device), Impressions_tensor.to(device), Impressions_len.to(device), Clicked.to(device)

            #yield User_en, Category, Subcategory, History_tensor, history_len, Category_Impressions, Subcategory_Impressions, Impressions_tensor, Impressions_len, Clicked



In [5]:
# Load Model
from LSTUR import LSTUR_con
from torch import nn,optim
device = "cpu"

LSTUR_con_module = LSTUR_con(
    seq_len = max_history_length,
    user_dim=300,
    user_size=User_vocab.__len__(),
    topic_size=Category_vocab.__len__(),
    topic_dim=100,
    subtopic_size=Subcategory_vocab.__len__(),
    subtopic_dim=100,
    word_dim=300,
    device=device
)

In [20]:
from sklearn.metrics import roc_auc_score
import numpy as np
Softmax = nn.Softmax(dim=1)
Softmax2 = nn.Softmax(dim=0)
labels = np.arange(0,295)

BatchLoader = load_batch(UserData, batch_size=10,train=False,device=device)


In [38]:
User_en, Category, Subcategory, History_tensor, history_len, Category_Impressions, Subcategory_Impressions, Impressions_tensor, Impressions_len, Clicked = BatchLoader.__next__()

y_true = th.empty(0)
preds = th.empty((0,295))
for i in range(10):
    output = LSTUR_con_module(User_en, Category, Subcategory, History_tensor, history_len, Category_Impressions, Subcategory_Impressions, Impressions_tensor)

    output = Softmax(output)
    preds = th.cat((preds,output),0)
    y_true = th.cat((y_true,Clicked),0)

    print(roc_auc_score(y_true.detach(),preds.detach(),multi_class="ovo",labels=np.arange(0,295)))


0.43452380952380953
0.43452380952380953
0.43452380952380953
0.43452380952380953
0.43452380952380953
0.43452380952380953
0.43452380952380953
0.43452380952380953
0.43452380952380953
0.43452380952380953


In [39]:
y_true.shape

torch.Size([100])

In [40]:
preds.shape

torch.Size([100, 295])

In [41]:
y_true

tensor([ 6., 17.,  1.,  6., 16.,  1.,  7.,  2., 18.,  7.,  6., 17.,  1.,  6.,
        16.,  1.,  7.,  2., 18.,  7.,  6., 17.,  1.,  6., 16.,  1.,  7.,  2.,
        18.,  7.,  6., 17.,  1.,  6., 16.,  1.,  7.,  2., 18.,  7.,  6., 17.,
         1.,  6., 16.,  1.,  7.,  2., 18.,  7.,  6., 17.,  1.,  6., 16.,  1.,
         7.,  2., 18.,  7.,  6., 17.,  1.,  6., 16.,  1.,  7.,  2., 18.,  7.,
         6., 17.,  1.,  6., 16.,  1.,  7.,  2., 18.,  7.,  6., 17.,  1.,  6.,
        16.,  1.,  7.,  2., 18.,  7.,  6., 17.,  1.,  6., 16.,  1.,  7.,  2.,
        18.,  7.])

In [42]:
preds.argmax(dim=1)

tensor([ 5, 18,  1,  7, 24,  4,  6,  5, 30, 15,  5, 18,  1, 10, 24,  4,  6,  5,
        21, 15,  5, 18,  1, 10, 24,  4,  6,  5, 21, 15,  5, 18,  1, 10, 24,  4,
         6,  5, 21, 15,  5, 18,  1, 10, 24,  4,  6,  5, 21, 15,  5, 18,  1,  7,
        24,  4,  6,  5, 21, 15,  5, 18,  1, 10, 24,  4,  6,  5, 21, 15,  5, 18,
         1, 10, 24,  4,  6,  5, 21, 15,  5, 18,  1, 10, 24,  4,  6,  5, 21, 15,
         5, 18,  1, 10, 24,  4,  6,  5, 21, 15])

In [24]:
for i in range(10):
    labels = np.arange(0,Impressions_len[i])

    probs = output[i][:Impressions_len[i]]
    probs = Softmax2(probs)

    score = roc_auc_score(Clicked[i],probs.reshape(-1,1),multi_class='ovo',labels=labels)   
    

TypeError: Singleton array array(93) cannot be considered a valid collection.

In [18]:
Softmax(output.cpu().detach())[0]

tensor([9.4204e-01, 1.0898e-11, 1.1135e-11, 1.1766e-12, 1.0373e-10, 1.0764e-11,
        1.4769e-08, 6.7888e-15, 3.7516e-10, 8.6651e-21, 1.3959e-02, 1.0878e-11,
        2.0267e-12, 1.7289e-06, 1.0610e-11, 2.7515e-02, 1.6191e-06, 2.4694e-03,
        2.6912e-17, 1.4003e-02, 1.0098e-10, 3.7744e-10, 3.4400e-08, 3.4051e-08,
        3.4408e-08, 3.4332e-08, 3.4264e-08, 3.4118e-08, 3.4378e-08, 3.4197e-08,
        3.4400e-08, 3.4174e-08, 3.4278e-08, 3.4335e-08, 3.4385e-08, 3.4212e-08,
        3.4237e-08, 3.4116e-08, 3.4138e-08, 3.4199e-08, 3.4400e-08, 3.3857e-08,
        3.4236e-08, 3.4153e-08, 3.4257e-08, 3.4162e-08, 3.4271e-08, 3.4096e-08,
        3.4128e-08, 3.4290e-08, 3.3993e-08, 3.4111e-08, 3.4159e-08, 3.4301e-08,
        3.4265e-08, 3.4082e-08, 3.4025e-08, 3.4189e-08, 3.4333e-08, 3.4247e-08,
        3.4100e-08, 3.4199e-08, 3.4150e-08, 3.4372e-08, 3.4351e-08, 3.4265e-08,
        3.4282e-08, 3.4147e-08, 3.4199e-08, 3.4037e-08, 3.4121e-08, 3.4073e-08,
        3.4046e-08, 3.4313e-08, 3.4202e-

In [105]:
take= np.array([0,2,0])

#take = [(0,1),(1,2),(2,2)]
g = np.array([[1,3,2],[4,5,6],[8,11,7]])


order = np.argsort(g,axis=1).argsort(axis=1)
order_test = np.argsort(g,axis=1)[::-1]
#print(order)

ranks = np.array([np.take(order[i],take[i]) for i in range(3)])

(1/(3-ranks)).mean()


0.611111111111111